In [ ]:
import os
import math
import random
import pickle

import yaml
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torch.utils import data
from torch.autograd import Variable
from torch.nn.utils import weight_norm
from torch.distributions.normal import Normal

In [ ]:
def from_npz_cache(file_path: str):
  npz = np.load(file_path, allow_pickle=True)
  return npz['observations'], npz['obs_speed'], npz['targets'], npz[
      'target_speed'], npz['mean'], npz['std']

In [ ]:
def download_assets():
  if not os.path.exists('/content/optimal.yaml'):
    !wget -P /content https://raw.githubusercontent.com/HarshayuGirase/PECNet/master/config/optimal.yaml

  if not os.path.exists('/content/eth_test.npz'):
    !wget --no-check-certificate -r 'https://docs.google.com/uc?export=download&id=1QvGR2cyduaO2kffcEtINlobS7cG5UBWp' -O eth_test.npz

  if not os.path.exists('/content/eth_train.npz'):
    !wget --no-check-certificate -r 'https://docs.google.com/uc?export=download&id=14m6fQNddxsomfDQBmcNew06j-PVsOAsx' -O eth_train.npz
  

download_assets()

--2020-10-07 22:04:25--  https://raw.githubusercontent.com/HarshayuGirase/PECNet/master/config/optimal.yaml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 600 [text/plain]
Saving to: ‘/content/optimal.yaml’

optimal.yaml        100%[===================>]     600  --.-KB/s    in 0s      

2020-10-07 22:04:26 (30.5 MB/s) - ‘/content/optimal.yaml’ saved [600/600]

will be placed in the single file you specified.

--2020-10-07 22:04:26--  https://docs.google.com/uc?export=download&id=1QvGR2cyduaO2kffcEtINlobS7cG5UBWp
Resolving docs.google.com (docs.google.com)... 74.125.195.102, 74.125.195.139, 74.125.195.113, ...
Connecting to docs.google.com (docs.google.com)|74.125.195.102|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: 

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Model Definitions

In [ ]:
class MLP(nn.Module):
  def __init__(self, input_dim, output_dim, hidden_size=(1024, 512), activation='relu', discrim=False, dropout=-1):
    super(MLP, self).__init__()
    dims = []
    dims.append(input_dim)
    dims.extend(hidden_size)
    dims.append(output_dim)
    self.layers = nn.ModuleList()
    for i in range(len(dims)-1):
        self.layers.append(nn.Linear(dims[i], dims[i+1]))

    if activation == 'relu':
        self.activation = nn.ReLU()
    elif activation == 'sigmoid':
        self.activation = nn.Sigmoid()

    self.sigmoid = nn.Sigmoid() if discrim else None
    self.dropout = dropout

  def forward(self, x):
    for i in range(len(self.layers)):
        x = self.layers[i](x)
        if i != len(self.layers)-1:
            x = self.activation(x)
            if self.dropout != -1:
                x = nn.Dropout(min(0.1, self.dropout/3) if i == 1 else self.dropout)(x)
        elif self.sigmoid:
            x = self.sigmoid(x)
    return x

In [ ]:
class PECNet(nn.Module):
  def __init__(self, 
               enc_past_size, 
               enc_dest_size, 
               enc_latent_size, 
               dec_size, 
               predictor_size, 
               fdim, 
               zdim, 
               sigma, 
               past_length, 
               future_length, 
               verbose):
    '''
    Args:
        size parameters: Dimension sizes
        sigma: Standard deviation used for sampling N(0, sigma)
        past_length: Length of past history (number of timesteps)
        future_length: Length of future trajectory to be predicted
    '''
    super(PECNet, self).__init__()

    self.zdim = zdim
    self.sigma = sigma

    # takes in the past
    self.encoder_past = MLP(input_dim = past_length*2, output_dim = fdim, hidden_size=enc_past_size)

    self.encoder_dest = MLP(input_dim = 2, output_dim = fdim, hidden_size=enc_dest_size)

    self.encoder_latent = MLP(input_dim = 2*fdim, output_dim = 2*zdim, hidden_size=enc_latent_size)

    self.decoder = MLP(input_dim = fdim + zdim, output_dim = 2, hidden_size=dec_size)

    self.predictor = MLP(input_dim = 2*fdim, output_dim = 2*(future_length-1), hidden_size=predictor_size)

    architecture = lambda net: [l.in_features for l in net.layers] + [net.layers[-1].out_features]

    if verbose:
        print("Past Encoder architecture : {}".format(architecture(self.encoder_past)))
        print("Dest Encoder architecture : {}".format(architecture(self.encoder_dest)))
        print("Latent Encoder architecture : {}".format(architecture(self.encoder_latent)))
        print("Decoder architecture : {}".format(architecture(self.decoder)))
        print("Predictor architecture : {}".format(architecture(self.predictor)))

  def forward(self, x, dest = None, device=torch.device('cpu')):

    # provide destination iff training
    # assert model.training
    assert self.training ^ (dest is None)
    
    # encode
    ftraj = self.encoder_past(x)

    if not self.training:
        z = torch.Tensor(x.size(0), self.zdim)
        z.normal_(0, self.sigma)

    else:
        # during training, use the destination to produce generated_dest and use it again to predict final future points

        # CVAE code
        dest_features = self.encoder_dest(dest)
        features = torch.cat((ftraj, dest_features), dim = 1)
        latent =  self.encoder_latent(features)

        mu = latent[:, 0:self.zdim] # 2-d array
        logvar = latent[:, self.zdim:] # 2-d array

        var = logvar.mul(0.5).exp_()
        eps = torch.DoubleTensor(var.size()).normal_()
        eps = eps.to(device)
        z = eps.mul(var).add_(mu)

    z = z.double().to(device)
    decoder_input = torch.cat((ftraj, z), dim = 1)
    generated_dest = self.decoder(decoder_input)

    if self.training:
        generated_dest_features = self.encoder_dest(generated_dest)

        prediction_features = torch.cat((ftraj, generated_dest_features), dim = 1)

        pred_future = self.predictor(prediction_features)
        return generated_dest, mu, logvar, pred_future

    return generated_dest

  # separated for forward to let choose the best destination
  # def predict(self, past, generated_dest, mask, initial_pos):
  def predict(self, past, generated_dest):
    ftraj = self.encoder_past(past)
    generated_dest_features = self.encoder_dest(generated_dest)

    prediction_features = torch.cat((ftraj, generated_dest_features), dim = 1)

    interpolated_future = self.predictor(prediction_features)
    return interpolated_future

## Loss

In [ ]:
def calculate_loss(x, reconstructed_x, mean, log_var, criterion, future, interpolated_future):
	# reconstruction loss
	RCL_dest = criterion(x, reconstructed_x)

	ADL_traj = criterion(future, interpolated_future) # better with l2 loss

	# kl divergence loss
	KLD = -0.5 * torch.sum(1 + log_var - mean.pow(2) - log_var.exp())

	return RCL_dest, KLD, ADL_traj

## Dataset

In [ ]:
class SocialDataset(data.Dataset):
	def __init__(self, npz_path, set_name="train", id=False, verbose=True):
		observations, _, targets, _, _, _ = from_npz_cache(npz_path)
		self.traj = np.concatenate([observations, targets], axis=1)
	
	def __len__(self):
		return len(self.traj)	

	def __getitem__(self, idx):
		return self.traj[idx]	


## Training

In [ ]:
def load_hyper_parameters(file_name='optimal.yaml'):
  with open(os.path.join('/content', file_name), 'r') as file:
    hyper_params = yaml.load(file)

  return hyper_params

In [ ]:
hyper_params = load_hyper_parameters()

In [ ]:
def train(train_dataset, model, optimizer):

	dataloader = data.DataLoader(
			train_dataset, batch_size=100, shuffle=True, num_workers=0)

	model.train()
	train_loss = 0
	total_rcl, total_kld, total_adl = 0, 0, 0
	criterion = nn.MSELoss()

	for i, trajx in enumerate(dataloader):

		traj = trajx - trajx[:, :1, :]
		traj *= hyper_params["data_scale"]		

		traj = torch.DoubleTensor(traj).to(device)
		x = traj[:, :hyper_params['past_length'], :]
		y = traj[:, hyper_params['past_length']:, :]

		x = x.view(-1, x.shape[1]*x.shape[2]) # (x,y,x,y ... )
		x = x.to(device)
		dest = y[:, -1, :].to(device)
		future = y[:, :-1, :].view(y.size(0),-1).to(device)

		# dest_recon, mu, var, interpolated_future = model.forward(x, initial_pos, dest=dest, mask=mask, device=device)
		dest_recon, mu, var, interpolated_future = model.forward(x, dest=dest, device=device)

		optimizer.zero_grad()
		rcl, kld, adl = calculate_loss(dest, dest_recon, mu, var, criterion, future, interpolated_future)
		loss = rcl + kld*hyper_params["kld_reg"] + adl*hyper_params["adl_reg"]
		loss.backward()

		train_loss += loss.item()
		total_rcl += rcl.item()
		total_kld += kld.item()
		total_adl += adl.item()
		optimizer.step()

	return train_loss, total_rcl, total_kld, total_adl

In [ ]:
def test(test_dataset, model, best_of_n = 1):
	'''Evalutes test metrics. Assumes all test data is in one batch'''

	dataloader = data.DataLoader(
			test_dataset, batch_size=len(test_dataset), shuffle=False, num_workers=0)

	model.eval()
	assert best_of_n >= 1 and type(best_of_n) == int

	with torch.no_grad():
		for i, trajx in enumerate(dataloader):

			traj = trajx - trajx[:, :1, :]
			traj *= hyper_params["data_scale"]			

			traj = torch.DoubleTensor(traj).to(device)
			x = traj[:, :hyper_params['past_length'], :]
			y = traj[:, hyper_params['past_length']:, :]
			y = y.cpu().numpy()

			# reshape the data
			x = x.view(-1, x.shape[1]*x.shape[2])
			x = x.to(device)

			dest = y[:, -1, :]
			all_l2_errors_dest = []
			all_guesses = []
			for _ in range(best_of_n):
				# dest_recon = model.forward(x, initial_pos, device=device)
				dest_recon = model.forward(x, device=device)
				dest_recon = dest_recon.cpu().numpy()
				all_guesses.append(dest_recon)

				l2error_sample = np.linalg.norm(dest_recon - dest, axis = 1)
				all_l2_errors_dest.append(l2error_sample)

			all_l2_errors_dest = np.array(all_l2_errors_dest)
			all_guesses = np.array(all_guesses)
			# average error
			l2error_avg_dest = np.mean(all_l2_errors_dest)

			# choosing the best guess
			indices = np.argmin(all_l2_errors_dest, axis = 0)

			best_guess_dest = all_guesses[indices,np.arange(x.shape[0]),  :]

			# taking the minimum error out of all guess
			l2error_dest = np.mean(np.min(all_l2_errors_dest, axis = 0))

			best_guess_dest = torch.DoubleTensor(best_guess_dest).to(device)

			interpolated_future = model.predict(x, best_guess_dest)
      # interpolated_future = interpolated_future.cpu().numpy()
			interpolated_future = interpolated_future.cpu().numpy()
			best_guess_dest = best_guess_dest.cpu().numpy()
	 
	 		# final overall prediction
			predicted_future = np.concatenate((interpolated_future, best_guess_dest), axis = 1)
			predicted_future = np.reshape(predicted_future, (-1, hyper_params['future_length'], 2)) # making sure
			# ADE error
			l2error_overall = np.mean(np.linalg.norm(y - predicted_future, axis = 2))

			l2error_overall /= hyper_params["data_scale"]
			l2error_dest /= hyper_params["data_scale"]
			l2error_avg_dest /= hyper_params["data_scale"]

			print('Test time error in destination best: {:0.3f} and mean: {:0.3f}'.format(l2error_dest, l2error_avg_dest))
			print('Test time error overall (ADE) best: {:0.3f}'.format(l2error_overall))

	return l2error_overall, l2error_dest, l2error_avg_dest

            

In [ ]:
def run_training():

  model = PECNet(
      hyper_params["enc_past_size"],
      hyper_params["enc_dest_size"],
      hyper_params["enc_latent_size"],
      hyper_params["dec_size"],
      hyper_params["predictor_hidden_size"],
      hyper_params["fdim"],
      hyper_params["zdim"],
      hyper_params["sigma"],
      hyper_params["past_length"],
      hyper_params["future_length"], verbose=True)
  
  model = model.double().to(device)
  optimizer = optim.Adam(model.parameters(), lr= hyper_params["learning_rate"])

  train_dataset = SocialDataset(
      '/content/eth_train.npz',
      set_name="train",
      verbose=True)
  
  test_dataset = SocialDataset(
      '/content/eth_test.npz',
      set_name="test",
      verbose=True)
  
  best_test_loss = 50 # start saving after this threshold
  best_endpoint_loss = 50
  N = hyper_params["n_values"]

  for e in range(hyper_params['num_epochs']):  
    train_loss, rcl, kld, adl = train(train_dataset, model,optimizer)
    test_loss, final_point_loss_best, final_point_loss_avg = test(test_dataset, model, best_of_n = N)

    if best_test_loss > test_loss:
      print("Epoch: ", e)
      print('################## BEST PERFORMANCE {:0.2f} ########'.format(test_loss))
      best_test_loss = test_loss
      if best_test_loss < 10.25:
        save_path = '/content/trained.pt'
        torch.save({
              'hyper_params': hyper_params,
              'model_state_dict': model.state_dict(),
              'optimizer_state_dict': optimizer.state_dict()
              }, save_path)
        print("Saved model to:\n{}".format(save_path))

    if final_point_loss_best < best_endpoint_loss:
      best_endpoint_loss = final_point_loss_best

    print("Train Loss", train_loss)
    print("RCL", rcl)
    print("KLD", kld)
    print("ADL", adl)
    print("Test ADE", test_loss)
    print("Test Average FDE (Across  all samples)", final_point_loss_avg)
    print("Test Min FDE", final_point_loss_best)
    print("Test Best ADE Loss So Far (N = {})".format(N), best_test_loss)
    print("Test Best Min FDE (N = {})".format(N), best_endpoint_loss)

In [ ]:
hyper_params['num_epochs'] = 100
hyper_params["data_scale"] = 170.

run_training()

Past Encoder architecture : [16, 512, 256, 16]
Dest Encoder architecture : [2, 8, 16, 16]
Latent Encoder architecture : [32, 8, 50, 32]
Decoder architecture : [32, 1024, 512, 1024, 2]
Predictor architecture : [32, 1024, 512, 256, 22]
(30307, 8, 2)
(30307, 12, 2)
(30307, 20, 2)
(364, 8, 2)
(364, 12, 2)
(364, 20, 2)
Test time error in destination best: 1.859 and mean: 2.044
Test time error overall (ADE) best: 1.012
Epoch:  0
################## BEST PERFORMANCE 1.01 ########
Saved model to:
/content/trained.pt
Train Loss 34612243.75906248
RCL 21301493.53796042
KLD 2653148.7481085635
ADL 10657601.472993527
Test ADE 1.0117048493366219
Test Average FDE (Across  all samples) 2.044313183439747
Test Min FDE 1.8589745748486588
Test Best ADE Loss So Far (N = 20) 1.0117048493366219
Test Best Min FDE (N = 20) 1.8589745748486588
Test time error in destination best: 1.464 and mean: 1.886
Test time error overall (ADE) best: 0.829
Epoch:  1
################## BEST PERFORMANCE 0.83 ########
Saved model 

## Test Pretrained Model

In [ ]:
checkpoint = torch.load('/content/trained.pt', map_location=device)
hyper_params = checkpoint["hyper_params"]  

In [ ]:
hyper_params["data_scale"]

170.0

In [ ]:
def test_model(test_dataset, model, best_of_n = 1):

	dataloader = data.DataLoader(
			test_dataset, batch_size=len(test_dataset), shuffle=False, num_workers=0)

	model.eval()
	assert best_of_n >= 1 and type(best_of_n) == int
	test_loss = 0

	with torch.no_grad():
		for i, trajx in enumerate(dataloader):

			traj = trajx - trajx[:, :1, :]
			traj *= hyper_params["data_scale"]

			traj = torch.DoubleTensor(traj).to(device)
	 
			x = traj[:, :hyper_params["past_length"], :]
			y = traj[:, hyper_params["past_length"]:, :]
			y = y.cpu().numpy()
	 
	 		# reshape the data
			x = x.view(-1, x.shape[1]*x.shape[2])
	 		# x = x.to(device)
			x = x.to(device)
		
			future = y[:, :-1, :]
			dest = y[:, -1, :]
			all_l2_errors_dest = []
			all_guesses = []
			for index in range(best_of_n):

				# dest_recon = model.forward(x, initial_pos, device=device)
				dest_recon = model.forward(x, device=device)
				dest_recon = dest_recon.cpu().numpy()
				all_guesses.append(dest_recon)

				l2error_sample = np.linalg.norm(dest_recon - dest, axis = 1)
				all_l2_errors_dest.append(l2error_sample)

			all_l2_errors_dest = np.array(all_l2_errors_dest)
			all_guesses = np.array(all_guesses)
			# average error
			l2error_avg_dest = np.mean(all_l2_errors_dest)

			# choosing the best guess
			indices = np.argmin(all_l2_errors_dest, axis = 0)

			best_guess_dest = all_guesses[indices,np.arange(x.shape[0]),  :]

			# taking the minimum error out of all guess
			l2error_dest = np.mean(np.min(all_l2_errors_dest, axis = 0))

			# back to torch land
			best_guess_dest = torch.DoubleTensor(best_guess_dest).to(device)

			# using the best guess for interpolation
			# interpolated_future = model.predict(x, best_guess_dest, mask, initial_pos)
			interpolated_future = model.predict(x, best_guess_dest)
			interpolated_future = interpolated_future.cpu().numpy()
			best_guess_dest = best_guess_dest.cpu().numpy()

			# final overall prediction
			predicted_future = np.concatenate((interpolated_future, best_guess_dest), axis = 1)
			predicted_future = np.reshape(predicted_future, (-1, hyper_params["future_length"], 2))

			# ADE error
			l2error_overall = np.mean(np.linalg.norm(y - predicted_future, axis = 2))

			l2error_overall /= hyper_params["data_scale"]
			l2error_dest /= hyper_params["data_scale"]
			l2error_avg_dest /= hyper_params["data_scale"]

			print('Test time error in destination best: {:0.3f} and mean: {:0.3f}'.format(l2error_dest, l2error_avg_dest))
			print('Test time error overall (ADE) best: {:0.3f}'.format(l2error_overall))

	return l2error_overall, l2error_dest, l2error_avg_dest

In [ ]:
def run_test(N=20):
  
  model = PECNet(
      hyper_params["enc_past_size"],
      hyper_params["enc_dest_size"],
      hyper_params["enc_latent_size"],
      hyper_params["dec_size"],
      hyper_params["predictor_hidden_size"],
      hyper_params["fdim"], 
      hyper_params["zdim"], 
      hyper_params["sigma"], 
      hyper_params["past_length"], 
      hyper_params["future_length"], verbose=True)
  
  
  model = model.double().to(device)
  model.load_state_dict(checkpoint["model_state_dict"])
  test_dataset = SocialDataset(
      "/content/eth_test.npz",
      set_name="test",
      verbose=True)
    
  #average ade/fde for k=20 (to account for variance in sampling)
  num_samples = 150
  average_ade, average_fde = 0, 0
  for i in range(num_samples):
    test_loss, final_point_loss_best, final_point_loss_avg = test_model(test_dataset, model, best_of_n = N)
    average_ade += test_loss
    average_fde += final_point_loss_best
    
  print()
  print("Average ADE:", average_ade/num_samples)
  print("Average FDE:", average_fde/num_samples)

In [ ]:
run_test()

Past Encoder architecture : [16, 512, 256, 16]
Dest Encoder architecture : [2, 8, 16, 16]
Latent Encoder architecture : [32, 8, 50, 32]
Decoder architecture : [32, 1024, 512, 1024, 2]
Predictor architecture : [32, 1024, 512, 256, 22]
(364, 8, 2)
(364, 12, 2)
(364, 20, 2)
Test time error in destination best: 0.980 and mean: 2.444
Test time error overall (ADE) best: 0.606
Test time error in destination best: 1.006 and mean: 2.435
Test time error overall (ADE) best: 0.613
Test time error in destination best: 0.970 and mean: 2.416
Test time error overall (ADE) best: 0.604
Test time error in destination best: 0.949 and mean: 2.432
Test time error overall (ADE) best: 0.600
Test time error in destination best: 0.929 and mean: 2.450
Test time error overall (ADE) best: 0.598
Test time error in destination best: 0.983 and mean: 2.444
Test time error overall (ADE) best: 0.618
Test time error in destination best: 0.967 and mean: 2.428
Test time error overall (ADE) best: 0.605
Test time error in de